# Importing required libraries

In [3]:
# Data analysis libraries
import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Database libraries
import pymongo

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

 # Obtaining data from database

In [ ]:
# config
DATABASE_URL = "mongodb://localhost:27017/"

MISSIONS = ["Mission 1 ","Mission 2", "Mission 3"]

In [5]:
client = MongoClient(DATABASE_URL)
training_db = client.training

# Pull training data
mission = db.reviews.find_one("Mission" : MISSIONS[0]) # Obtain data with this tag




# Heart Rate
def query_heart():
    pass

# Temp
def query_temp():
    pass

# Altitude
def query_altitude():
    pass

# Location
def query_location():
    pass

# Model Training

In this section, we will collect and monitor the firefighter trainee performance
This will be used for the initial training

In [ ]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

# References

https://www.mongodb.com/blog/post/getting-started-with-python-and-mongodb